The [last notebook](https://atmos.washington.edu/~nbren12/reports/uwnet/notebooks/papers/boostrap-lrfs.html) shows that the LRFs varied a lot accross a boostrap sample taken from the tropics. I guess that this is due to diurnal variability. In this notebook, I study the diurnal cycle of the LRF, and the robustness of the upper atmospheric sensitivity.


In [ ]:
from jacobian import *
from src.data import open_data
import common

In [ ]:
model = get_model(path="../../models/265/5.pkl")
training = open_data('training')
training['region'] = common.get_regions(training.y)
training['time_of_day'] = (training.time % 1) * 24
tropics = training.isel(y=slice(30,34)).load()

# Diurnal cycle of the LRF

In [ ]:
diurnal_mean = {time: ds.mean(['x', 'y', 'time']).expand_dims(['x', 'y'], [-1, -2]) for time, ds in tropics.groupby('time_of_day')}
jacobians = {time: get_jacobian(model, sample) for time, sample in diurnal_mean.items()}

In [ ]:
p = tropics.p[0].values

In [ ]:
for key, jac in jacobians.items():
    plot((jac, p))
    plt.suptitle(f'Time of day = {key}')

The minimum value of each element of the LRFs is stable accross the diurnal cycle:

In [ ]:
q025 = apply_list_jacobian(quantile(q=.025), list(jacobians.values()))
plot((q025, p));

The LRFs are actually quite consistent accross the diurnal cycle. It must be some other factor which explains the variance in the LRFs. My next guess is that PW explains the non-linearity of the LRF.